<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_6/S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 16,kernel_size=3, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 16,kernel_size=3, bias=False)
        self.bn2 = nn.BatchNorm2d(16)
        
        self.conv3 = nn.Conv2d(16, 16,kernel_size=3, bias=False)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.conv4 = nn.Conv2d(16, 16, kernel_size=3, bias=False)
        self.bn4 = nn.BatchNorm2d(16)

        self.conv5 = nn.Conv2d(16, 16, kernel_size=3, bias=False)
        self.bn5 = nn.BatchNorm2d(16)

        self.conv6 = nn.Conv2d(16, 16, kernel_size=3, bias=False)
        self.bn6 = nn.BatchNorm2d(16)

        self.conv7 = nn.Conv2d(16, 16, kernel_size=3, bias=False)
        self.bn7 = nn.BatchNorm2d(16)

        self.conv8 = nn.Conv2d(16,10, kernel_size=1, bias=False)
        self.bn8 = nn.BatchNorm2d(10)
        
        self.gap = nn.AvgPool2d(4)

    def forward(self, x):
        x = F.relu(self.conv1(x)) # C1
        x = self.bn1(x) # BN1

        x = F.relu(self.conv2(x)) # C2
        x = self.bn2(x) # BN2
        
        x = F.relu(self.conv3(x)) # C3
        x = self.bn3(x) # BN3
        
        x = F.relu(self.conv4(x)) # C4
        x = self.bn4(x) # BN4

        x = F.relu(F.max_pool2d(x,2)) # MP1

        x = F.relu(self.conv5(x)) # C5 
        x = self.bn5(x) # BN5

        x = F.relu(self.conv6(x)) # C6
        x = self.bn6(x) # BN6

        x = F.relu(self.conv7(x)) # C7 
        x = self.bn7(x) # BN7

        x = F.relu(self.conv8(x)) # C8 
        x = self.bn8(x) # BN8


        x = self.gap(x)
        x = x.view(-1,10)
        return F.log_softmax(x,dim=1)

In [18]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,304
       BatchNorm2d-4           [-1, 16, 24, 24]              32
            Conv2d-5           [-1, 16, 22, 22]           2,304
       BatchNorm2d-6           [-1, 16, 22, 22]              32
            Conv2d-7           [-1, 16, 20, 20]           2,304
       BatchNorm2d-8           [-1, 16, 20, 20]              32
            Conv2d-9             [-1, 16, 8, 8]           2,304
      BatchNorm2d-10             [-1, 16, 8, 8]              32
           Conv2d-11             [-1, 16, 6, 6]           2,304
      BatchNorm2d-12             [-1, 16, 6, 6]              32
           Conv2d-13             [-1, 16, 4, 4]           2,304
      BatchNorm2d-14             [-1, 1

In [19]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [20]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch, train_acc, train_losses):
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss+=loss.item()
        loss.backward()
        optimizer.step()
        correct+= output.argmax(dim=1).eq(target).sum().item()
        processed+= len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy = {100*correct/processed:0.2f}')

    train_acc.append(100*correct/processed)
    train_losses.append(train_loss/len(train_loader))


def test(model, device, test_loader, test_acc, test_losses):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc.append(100.*correct/len(test_loader.dataset))

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [21]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)
train_losses = []
test_losses = []
train_acc = []
test_acc = []
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch, train_acc, train_losses)
    test(model, device, test_loader, test_acc, test_losses)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.


loss=0.08233136683702469 batch_id=468 Accuracy = 95.09: 100%|██████████| 469/469 [00:19<00:00, 24.06it/s]



Test set: Average loss: 0.0747, Accuracy: 9876/10000 (98.76%)

Adjusting learning rate of group 0 to 1.0000e-02.


loss=0.04755251482129097 batch_id=468 Accuracy = 98.51: 100%|██████████| 469/469 [00:19<00:00, 24.55it/s]



Test set: Average loss: 0.0538, Accuracy: 9891/10000 (98.91%)

Adjusting learning rate of group 0 to 1.0000e-02.


...


Test set: Average loss: 0.0191, Accuracy: 9945/10000 (99.45%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.0032746519427746534 batch_id=468 Accuracy = 99.83: 100%|██████████| 469/469 [00:19<00:00, 23.60it/s]



Test set: Average loss: 0.0195, Accuracy: 9943/10000 (99.43%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.00876711867749691 batch_id=468 Accuracy = 99.84: 100%|██████████| 469/469 [00:19<00:00, 23.89it/s]



Test set: Average loss: 0.0195, Accuracy: 9947/10000 (99.47%)

Adjusting learning rate of group 0 to 1.0000e-03.
